In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('/Users/toshiya/Workspace/lab/doc/keypoint_matching/result_flann_ts.csv')

In [3]:
df = df.drop('Unnamed: 0',axis=1)

In [7]:
df = df.sort_values(by='cow_id')
df.loc[:2,:]

,cow_id,top1,top3
0,cow010001,1.0,1.0
1,cow010002,1.0,1.0
2,cow010003,1.0,1.0


In [8]:
df.iloc[:2,:]

,cow_id,top1,top3
0,cow010001,1.0,1.0
1,cow010002,1.0,1.0


In [32]:
np.mean(df['top1'])

0.8945324167857144

In [33]:
np.mean(df['top3'])

0.9214153786785715

In [2]:
df_train = pd.read_csv('../processed_data/train_v12.csv')
df_test = pd.read_csv('../processed_data/test_v12.csv')

### 港区フラグ付与

In [3]:
tra_ku = pd.DataFrame({
    '23ku': df_train['23ku'].value_counts().index,
    'num': df_train['23ku'].value_counts()
})

In [4]:
tes_ku = pd.DataFrame({
    '23ku': df_test['23ku'].value_counts().index,
    'num': df_test['23ku'].value_counts()
})

In [5]:
pd.merge(tra_ku,tes_ku,on='23ku')

,23ku,num_x,num_y
0,世田谷区,3023,2948
1,足立区,2191,2173
2,大田区,2000,1965
3,杉並区,1769,1773
4,練馬区,1752,1770
5,江戸川区,1702,1697
6,板橋区,1623,1646
7,港区,1512,1560
8,江東区,1508,1538
9,中央区,1279,1253


- train,testで分布の差異はない

In [6]:
def minatoku_flag(train,test):
    n_tra = train.copy()
    n_tes = test.copy()
    
    n_tra['minato'] = np.where(np.array(n_tra['23ku'])=='港区',1,0)
    n_tes['minato'] = np.where(np.array(n_tes['23ku'])=='港区',1,0)
    
    return n_tra,n_tes

In [7]:
_a,_b = minatoku_flag(df_train,df_test)

In [8]:
_a[_a['23ku']=='港区'].loc[:,['23ku','minato']]

,23ku,minato
12,港区,1
68,港区,1
102,港区,1
107,港区,1
120,港区,1
...,...,...
31378,港区,1
31391,港区,1
31399,港区,1
31416,港区,1


### 23ku_mean_std の細分化

- 今までは２３区ごとの平均を標準化したものを用いていた
- 部屋数や間取りを考慮に入れて一人用かそれ以外でマッピングを変える必要がある気がした
- ３部屋以下を一人暮らしとする

In [9]:
tra = df_train.loc[:,['id','y','room_num','23ku']]
# tra[tra['id']==20428]

In [10]:
tra = df_train.loc[:,['id','y','room_num','23ku']]
tra.drop(tra[tra['id']==5776].index)
tra.drop(tra[tra['id']==20927].index)
tra.drop(tra[tra['id']==20232].index)
tra.drop(tra[tra['id']==20428].index)

,id,y,room_num,23ku
0,1,75000,1,北区
1,2,76000,1,中央区
2,3,110000,1,渋谷区
3,4,150000,2,杉並区
4,5,74000,2,葛飾区
...,...,...,...,...
31461,31466,80000,2,板橋区
31462,31467,54500,1,世田谷区
31463,31468,125000,3,江東区
31464,31469,98000,1,中野区


In [11]:
tra.groupby('room_num').describe()['y']

,count,mean,std,min,25%,50%,75%,max
room_num,,,,,,,,
1,24056.0,104949.380612,48230.179465,25000.0,72000.0,95000.0,125000.0,1203500.0
2,5561.0,146820.613739,95248.104639,33000.0,88000.0,120000.0,175000.0,1800000.0
3,1666.0,195535.000000,125121.700764,59000.0,120000.0,157000.0,230000.0,1300000.0
4,156.0,281617.307692,219900.329963,70000.0,173000.0,240000.0,300000.0,1660000.0
5,25.0,330424.000000,238176.953265,80000.0,170000.0,270000.0,375000.0,900000.0
6,2.0,350000.000000,141421.356237,250000.0,300000.0,350000.0,400000.0,450000.0


In [12]:
tra.groupby(by=['23ku','room_num']).describe()['y']

count           mean            std       min       25%  \
23ku room_num                                                             
世田谷区 1         2269.0   96649.779639   36445.477506   30000.0   68000.0   
     2          555.0  146402.216216   52376.454615   45000.0  110000.0   
     3          177.0  203328.813559   75399.925513  101100.0  149000.0   
     4           20.0  343000.000000  136008.126692  170000.0  265000.0   
     5            2.0  280000.000000       0.000000  280000.0  280000.0   
...               ...            ...            ...       ...       ...   
足立区  1         1490.0   78804.135570   26522.845369   35000.0   60000.0   
     2          549.0   94611.429872   32087.610899   40000.0   70000.0   
     3          134.0  117461.940299   29687.821342   65000.0   98000.0   
     4           14.0  172428.571429   55480.716381  115000.0  140000.0   
     5            4.0  158250.000000   63025.788373  108000.0  124500.0   

                    50%       75%       max  
23ku room_num                                
世田谷区 1          90000.0  119000.0  320000.0  
     2         133000.0  174000.0  326000.0  
     3         190000.0  245000.0  590000.0  
     4         290000.0  420000.0  690000.0  
     5         280000.0  280000.0  280000.0  
...                 ...       ...       ...  
足立区  1          73000.0   90000.0  210600.0  
     2          85800.0  115000.0  202000.0  
     3         114000.0  130000.0  219000.0  
     4         147000.0  227500.0  256000.0  
     5         137500.0  171250.0  250000.0  

[107 rows x 8 columns]

In [13]:
pd.DataFrame({
    '23ku_mean_2': tra.groupby(by=['23ku', 'room_num']).describe()['y']['mean'],
}).to_csv('../processed_data/23ku_mean_2.csv')

In [14]:
ku_mean_2 = pd.read_csv('../processed_data/23ku_mean_2.csv')
ku_mean_2

,23ku,room_num,23ku_mean_2
0,世田谷区,1,96649.779639
1,世田谷区,2,146402.216216
2,世田谷区,3,203328.813559
3,世田谷区,4,343000.000000
4,世田谷区,5,280000.000000
...,...,...,...
102,足立区,1,78804.135570
103,足立区,2,94611.429872
104,足立区,3,117461.940299
105,足立区,4,172428.571429


In [15]:
ku_mean_2[ku_mean_2['23ku']=='台東区']

,23ku,room_num,23ku_mean_2
22,台東区,1,119554.363412
23,台東区,2,181114.159292
24,台東区,3,176200.000000


- 6部屋のデータがない区が存在する
- 千代田区と台東区だけ4部屋以上のデータがない
- ３部屋以下はその平均値でいけそう
- 4部屋以上でも５部屋のデータが少なく４部屋の平均より安くなっていることがある

In [16]:
ku_mean_2.groupby('23ku').describe()['room_num']

,count,mean,std,min,25%,50%,75%,max
23ku,,,,,,,,
世田谷区,5.0,3.00,1.581139,1.0,2.00,3.0,4.00,5.0
中央区,4.0,2.50,1.290994,1.0,1.75,2.5,3.25,4.0
中野区,5.0,3.00,1.581139,1.0,2.00,3.0,4.00,5.0
北区,5.0,3.00,1.581139,1.0,2.00,3.0,4.00,5.0
千代田区,3.0,2.00,1.000000,1.0,1.50,2.0,2.50,3.0
台東区,3.0,2.00,1.000000,1.0,1.50,2.0,2.50,3.0
品川区,5.0,3.00,1.581139,1.0,2.00,3.0,4.00,5.0
墨田区,5.0,3.00,1.581139,1.0,2.00,3.0,4.00,5.0
大田区,5.0,3.00,1.581139,1.0,2.00,3.0,4.00,5.0


In [17]:
len(ku_mean_2[ku_mean_2['room_num']<5])

89

In [18]:
ku_mean_2 = ku_mean_2[ku_mean_2['room_num']<5].reset_index(drop=True)
ku_mean_2.to_csv('../processed_data/23ku_mean_2.csv')
# 千代田区と台東区だけ4部屋以上のデータがないのでそれぞれ分布が似ている
# 港区と江東区のデータで補完
ku_mean_2[ku_mean_2['23ku']=='千代田区']

,23ku,room_num,23ku_mean_2
16,千代田区,1,152027.353690
17,千代田区,2,320060.000000
18,千代田区,3,553357.142857


In [19]:
ku_mean_2[ku_mean_2['23ku']=='港区']

,23ku,room_num,23ku_mean_2
61,港区,1,169779.406430
62,港区,2,353491.596639
63,港区,3,482842.105263
64,港区,4,990000.000000


In [20]:
ku_mean_2[ku_mean_2['23ku']=='台東区']

,23ku,room_num,23ku_mean_2
19,台東区,1,119554.363412
20,台東区,2,181114.159292
21,台東区,3,176200.000000


In [21]:
ku_mean_2[ku_mean_2['23ku']=='江東区']

,23ku,room_num,23ku_mean_2
53,江東区,1,112954.473913
54,江東区,2,153816.886364
55,江東区,3,205574.117647
56,江東区,4,238287.500000


In [22]:
ku_mean_2[ku_mean_2['23ku']=='新宿区']

,23ku,room_num,23ku_mean_2
38,新宿区,1,116898.798521
39,新宿区,2,218169.090909
40,新宿区,3,293041.666667


In [25]:
ku_mean_2[ku_mean_2['23ku']=='品川区']

,23ku,room_num,23ku_mean_2
22,品川区,1,111640.905018
23,品川区,2,196878.048780
24,品川区,3,280366.666667
25,品川区,4,367500.000000


In [26]:
hokan = pd.DataFrame({
    '23ku':['千代田区','台東区','新宿区'],
    'room_num': [4,4,4],
    '23ku_mean_2':[990000.000000,238287.500000,367500.000000]
})

In [27]:
ku_mean_2 = pd.concat([ku_mean_2,hokan]).sort_values(['23ku','room_num']).reset_index(drop=True)

In [28]:
len(ku_mean_2)

92

In [29]:
ku_mean_2[ku_mean_2['room_num']==4]

,23ku,room_num,23ku_mean_2
3,世田谷区,4,343000.000000
7,中央区,4,185000.000000
11,中野区,4,300000.000000
15,北区,4,207500.000000
19,千代田区,4,990000.000000
23,台東区,4,238287.500000
27,品川区,4,367500.000000
31,墨田区,4,300000.000000
35,大田区,4,341352.941176
39,文京区,4,457200.000000


In [32]:
ku_mean_2.to_csv('../processed_data/23ku_mean_2.csv',index=False)

In [54]:
len(ku_mean_2)

92

In [65]:
def processing_location_2(train,test):
    df = pd.read_csv('../processed_data/23ku_mean_2.csv')
    
    res_tra = []
    res_tes = []
    for ku,rn in zip(train.loc[:,'23ku'],train.loc[:,'room_num']):
        if rn>4:
            rn = 4
        tmp = float(df[(df['23ku']==ku)&(df['room_num']==rn)]['23ku_mean_2'])
        res_tra.append(tmp)
        
    for ku,rn in zip(test.loc[:,'23ku'],test.loc[:,'room_num']):
        if rn>4:
            rn = 4
        tmp = float(df[(df['23ku']==ku)&(df['room_num']==rn)]['23ku_mean_2'])
        res_tes.append(tmp)
    
    return res_tra,res_tes

In [66]:
a, b = processing_location_2(df_train,df_test)

In [67]:
len(a),len(df_train)

(31466, 31466)

In [68]:
len(b),len(df_test)

(31262, 31262)